TensorBoard provides an interactive GUI for viewing data logs from machine-learning training jobs. However, users sometimes want to **programmatically** read the data logs stored in TensorBoard, for purposes such as performing custom analyses and creating custom visualizations of the log data.

A feature is introduced in TensorBoard v2.3 to support this use case:  `tensorboard.data.experimental.ExperimentFromDev()`. It allows prgorammatic access to TensorBoard's scalar logs.

Please note:
1. the API is still in its experimental stage, as reflected by its API namespace. 
2. this API currently supports only experimental uploaded to [tensorboard.dev](https://tensorboard.dev).   Support for locally stored TensorBoard logdir will be added in the future 